This file, once complete, will obtain a dataframe that contains every trade that occured during the 2019-20 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains every trade that occured in the league for the 2012-13 season. So I am going to scrape the cap friendly website (Trades tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed so that after the load more button is clicked and no more rows appear (once the page is opened in the selenium server), the table is then downloaded to a dataframe.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service= Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/trades?season=2013"
driver.get(url)

# Wait for the page to load and click "Load More" until all rows are loaded
while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            #EC.element_to_be_clickable((By.XPATH, "//button[text()='Load More']"))
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[10]/div/div/div[21]/div[1]"))
        )
        load_more_button.click()
        time.sleep(5)  # wait for the new rows to load
        # Check if "No results found" text is present
        no_results_text = driver.find_elements(By.XPATH, "/html/body/div[10]/div/div/div[20]/table/tbody/tr[154]/td/div")
        if no_results_text:
            break  # if "No results found" text is present, exit the loop
    except:
        break  # if no more "Load More" button, exit the loop

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]


C:\Users\nsofi\AppData\Local\Temp\ipykernel_17956\624532109.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
#print(dfattempt)
dfattempt.to_csv("Sel2013Att1.csv")

In [4]:
dfattempt.shape
dfattempt.tail(5)

DATE                                      TRADE DETAILS  \
                DATE                                      TRADE DETAILS   
91     Jul. 20, 2012  Florida Panthers Acquire:Casey Wellman · $0$80...   
92     Jul. 10, 2012  St. Louis Blues Acquire:2014 4th round pick (T...   
93      Jul. 2, 2012  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94      Jul. 1, 2012  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95  No Results Found                                   No Results Found   

                                                       \
                                      TRADE DETAILS.1   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       \
                                      TRADE DETAILS.2   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       \
                                      TRADE DETAILS.3   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       \
                                      TRADE DETAILS.4   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       \
                                      TRADE DETAILS.5   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       \
                                      TRADE DETAILS.6   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       \
                                      TRADE DETAILS.7   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...   
95                                   No Results Found   

                                                       ...  \
                                      TRADE DETAILS.8  ...   
91  Florida Panthers Acquire:Casey Wellman · $0$80...  ...   
92  St. Louis Blues Acquire:2014 4th round pick (T...  ...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000...  ...   
94  Columbus Blue Jackets Acquire:Nick Foligno · $...  ...   
95                                   No Results Found  ...   

                                                       \
                                     TRADE DETAILS.24   
91  New York Rangers Acquire:2014 5th round pick (...   
92  Tampa Bay Lightning Acquire:B.J. Crombeen · $1...   
93  Dallas Stars Acquire:De

BOOM!! Scraped all 79 rows I need dynamically using the selenium driver. Now do the data cleaning that I have worked on and come close to getting fully corrected on it. From there, make the last fixes manually. About 90% of getting a cleaned trades dataframe is automated with 10% being manual inside the csv file itslef.

In [5]:
dfattempt.head(5)

DATE                                      TRADE DETAILS  \
            DATE                                      TRADE DETAILS   
0  Jun. 30, 2013  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Jun. 30, 2013  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Jun. 30, 2013  Florida Panthers Acquire:2013 7th round pick (...   
3  Jun. 30, 2013  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Jun. 30, 2013  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.1   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.2   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.3   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.4   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.5   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.6   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      \
                                     TRADE DETAILS.7   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...   
2  Florida Panthers Acquire:2013 7th round pick (...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...   

                                                      ...  \
                                     TRADE DETAILS.8  ...   
0  Minnesota Wild Acquire:Benn Ferriero · $0$550,...  ...   
1  Columbus Blue Jackets Acquire:2013 2nd round p...  ...   
2  Florida Panthers Acquire:2013 7th round pick (...  ...   
3  Edmonton Oilers Acquire:2013 3rd round pick (S...  ...   
4  Chicago Blackhawks Acquire:2013 3rd round pick...  ...   

                                                      \
                                    TRADE DETAILS.24   
0  New York Rangers Acquire:Justin Falk · $975,00...   
1  Pittsburgh Penguins Acquire:2013 2nd round pic...   
2  Montreal Canadiens Acquire:2014 7th round pick...   
3  St. Louis Blues Acquire:2013 2nd round pick (L...   

In [6]:
df= pd.read_csv("Sel2013Att1.csv", header=1)
print(df)

    Unnamed: 0              DATE  \
0            0     Jun. 30, 2013   
1            1     Jun. 30, 2013   
2            2     Jun. 30, 2013   
3            3     Jun. 30, 2013   
4            4     Jun. 30, 2013   
..         ...               ...   
91          91     Jul. 20, 2012   
92          92     Jul. 10, 2012   
93          93      Jul. 2, 2012   
94          94      Jul. 1, 2012   
95          95  No Results Found   

                                        TRADE DETAILS  \
0   Minnesota Wild Acquire:Benn Ferriero · $0$550,...   
1   Columbus Blue Jackets Acquire:2013 2nd round p...   
2   Florida Panthers Acquire:2013 7th round pick (...   
3   Edmonton Oilers Acquire:2013 3rd round pick (S...   
4   Chicago Blackhawks Acquire:2013 3rd round pick...   
..                                                ...   
91  Florida Panthers Acquire:Casey Wellman · $0$80...   
92  St. Louis Blues Acquire:2014 4th round pick (T...   
93  Buffalo Sabres Acquire:Adam Pardy · $2,000,000... 

In [7]:
df.head(5)

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
0,0,"Jun. 30, 2013","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","Minnesota Wild Acquire:Benn Ferriero · $0$550,...",...,"New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00...","New York Rangers Acquire:Justin Falk · $975,00..."
1,1,"Jun. 30, 2013",Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,Columbus Blue Jackets Acquire:2013 2nd round p...,...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Pittsburgh Penguins Acquire:2013 2nd round pic...
2,2,"Jun. 30, 2013",Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,Florida Panthers Acquire:2013 7th round pick (...,...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...,Montreal Canadiens Acquire:2014 7th round pick...
3,3,"Jun. 30, 2013",Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,Edmonton Oilers Acquire:2013 3rd round pick (S...,...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,St. Louis Blues Acquire:2013 2nd round pick (L...,S

Alright so I can see that everthing I need got extracted into the dataframe which is awesome. All the information I need can get successfully pulled. However, the format is an absolute disaster. I need to filter what information is going into the df and how it is getting pulled. Just going to send to a csv file real quick to view it better.

Alright from looking at it, A ton of information gets repeated (Lot of repeat variables). SO, I just need to filter and then use some text splitting to get information where it needs to go in the dataframe. I am going to attempt to drop the repetive columns of the dataframe in the next code chunk. Since I am dropping all but two columns, I am simply going to make a new dataframe that contains two columns.

In [8]:
better_df= df[['DATE','TRADE DETAILS', 'TRADE DETAILS.18']]
better_df.head(5)
better_df.to_csv('All Correct Info2012-13.csv')

Now that all the repetitive columns are gone, I am going to split the good trade details column into multiple columns to make the dataframe much cleaner and easier to read. 

In [9]:
better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
better_df[['Team_y', 'y_receieved']] = better_df['TRADE DETAILS.18'].str.split('Acquire:', expand=True)
better_df.head(5)
better_df.to_csv('Close2012-13.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_17956\27607454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
C:\Users\nsofi\AppData\Local\Temp\ipykernel_17956\27607454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
C:\Users\nsofi\AppData\Local\Temp\ipykernel_17956\27607454.py:2: SettingWithCopyWarning:

In [10]:
#Have to re-read the Close.csv file back in without the last two rows which have nonsense in them. Making the edit straight in csv file and then just re-reading the csv file after
better_df=pd.read_csv("Close2012-13.csv")

In [11]:
# Define the function to split based on conditions
def conditional_splitx(x_receieved):
    if x_receieved.startswith('20') and "[Conditional]" not in x_receieved:
        parts = x_receieved.rsplit(')', 1)
        receivedx = parts[0] + ')'
        Monetary = parts[1] if len(parts) > 1 else ''
    elif x_receieved.startswith('20') and "[Conditional]" in x_receieved:
        parts = x_receieved.rsplit('Sum', 1)
        receivedx = parts[0] 
        Monetary = parts[1] if len(parts) > 1 else ''
    elif "Future Considerations" in x_receieved:
        parts =x_receieved.rsplit('Sum', 1)
        receivedx = parts[0] 
        Monetary = parts[1] if len(parts) > 1 else ''
    else:
        parts = x_receieved.split(' · ', 1)
        receivedx = parts[0]
        Monetary = ' · ' + parts[1] if len(parts) > 1 else ''
    return pd.Series([receivedx, Monetary])

# Apply the function and create new columns
better_df[['receivedx', 'Monetary']] = better_df['x_receieved'].apply(conditional_splitx)
#better_df[['X_received']] = better_df['receivedx'].str.split('Sum',expand=True)




better_df.head(5)
#better_df.to_csv('Progress2022-23.csv')


,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary
0,0,"Jun. 30, 2013","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","New York Rangers Acquire:Justin Falk · $975,00...",Minnesota Wild,"Benn Ferriero · $0$550,000$550,000 (AHL/JR)201...",New York Rangers,"Justin Falk · $975,000$975,000$975,000Sum: $97...",Benn Ferriero,"· $0$550,000$550,000 (AHL/JR)2014 6th round p..."
1,1,"Jun. 30, 2013",Columbus Blue Jackets Acquire:2013 2nd round p...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Columbus Blue Jackets,2013 2nd round pick (SJS - #50 - Dillon Heathe...,Pittsburgh Penguins,2013 2nd round pick (CBJ - #44 - Tristan Jarry...,2013 2nd round pick (SJS - #50 - Dillon Heathe...,Sum: $0$0$0Change: $0Change: $0Change: $0
2,2,"Jun. 30, 2013",Florida Panthers Acquire:2013 7th round pick (...,Montreal Canadiens Acquire:2014 7th round pick...,Florida Panthers,2013 7th round pick (MTL - #206 - MacKenzie We...,Montreal Canadiens,2014 7th round pick (FLA - #182 - Hugo Fagerbl...,2013 7th round pick (MTL - #206 - MacKenzie We...,Sum: $0$0$0Change: $0Change: $0Change: $0
3,3,"Jun. 30, 2013",Edmonton Oilers Acquire:2013 3rd round pick (S...,St. Louis Blues Acquire:2013 2nd round pick (L...,Edmonton Oilers,2013 3rd round pick (STL - #83 - Bogdan Yakimo...,St. Louis Blues,2013 2nd round pick (LAK - #57 - William Carri...,2013 3rd round pick (STL - #83 - Bogdan Yakimo...,Sum: $0$0$0Change: $0Change: $0Change: $0
4,4,"Jun. 30, 2013",Chicago Blackhawks Acquire:2013 3rd round pick...,"Winnipeg Jets Acquire:Michael Frolík · $2,333,...",Chicago Blackhawks,2013 3rd round pick (WPG - #74 - John Hayden)2...,Winnipeg Jets,"Michael Frolík · $2,333,333$2,333,333$1,900,00...",2013 3rd round pick (WPG - #74 - John Hayden)2...,"Sum: $0$0$0Change: -$2,333,333Change: -$2,333,..."


Just did a bunch of string manipulation to x_received, now have to do it to y_received

In [12]:
# Define the function to split based on conditions
def conditional_splity(y_receieved):
    if y_receieved.startswith('20') and "[Conditional]" not in y_receieved:
        parts = y_receieved.rsplit(')', 1)
        receivedy = parts[0] + ')'
        Monetary2 = parts[1] if len(parts) > 1 else ''
    elif y_receieved.startswith('20') and "[Conditional]" in y_receieved:
        parts =y_receieved.rsplit('Sum', 1)
        receivedy = parts[0] 
        Monetary2 = parts[1] if len(parts) > 1 else ''
    elif "Future Considerations" in y_receieved:
        parts =y_receieved.rsplit('Sum', 1)
        receivedy = parts[0] 
        Monetary2 = parts[1] if len(parts) > 1 else ''
    else:
        parts = y_receieved.split(' · ', 1)
        receivedy = parts[0]
        Monetary2 = ' · ' + parts[1] if len(parts) > 1 else ''
    return pd.Series([receivedy, Monetary2])

# Apply the function and create new columns
better_df[['receivedy', 'Monetary2']] = better_df['y_receieved'].apply(conditional_splity)
#better_df[['X_received']] = better_df['receivedx'].str.split('Sum',expand=True)




better_df.head(5)
#better_df.to_csv('Progress9.csv')

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary,receivedy,Monetary2
0,0,"Jun. 30, 2013","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","New York Rangers Acquire:Justin Falk · $975,00...",Minnesota Wild,"Benn Ferriero · $0$550,000$550,000 (AHL/JR)201...",New York Rangers,"Justin Falk · $975,000$975,000$975,000Sum: $97...",Benn Ferriero,"· $0$550,000$550,000 (AHL/JR)2014 6th round p...",Justin Falk,"· $975,000$975,000$975,000Sum: $975,000$975,0..."
1,1,"Jun. 30, 2013",Columbus Blue Jackets Acquire:2013 2nd round p...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Columbus Blue Jackets,2013 2nd round pick (SJS - #50 - Dillon Heathe...,Pittsburgh Penguins,2013 2nd round pick (CBJ - #44 - Tristan Jarry...,2013 2nd round pick (SJS - #50 - Dillon Heathe...,Sum: $0$0$0Change: $0Change: $0Change: $0,2013 2nd round pick (CBJ - #44 - Tristan Jarry),Sum: $0$0$0Change: $0Change: $0Change: $0
2,2,"Jun. 30, 2013",Florida Panthers Acquire:2013 7th round pick (...,Montreal Canadiens Acquire:2014 7th round pick...,Florida Panthers,2013 7th round pick (MTL - #206 - MacKenzie We...,Montreal Canadiens,2014 7th round pick (FLA - #182 - Hugo Fagerbl...,2013 7th round pick (MTL - #206 - MacKenzie We...,Sum: $0$0$0Change: $0Change: $0Change: $0,2014 7th round pick (FLA - #182 - Hugo Fagerblom),Sum: $0$0$0Change: $0Change: $0Change: $0
3,3,"Jun. 30, 2013",Edmonton Oilers Acquire:2013 3rd round pick (S...,St. Louis Blues Acquire:2013 2nd round pick (L...,Edmonton Oilers,2013 3rd round pick (STL - #83 - Bogdan Yakimo...,St. Louis Blues,2013 2nd round pick (LAK - #57 - William Carri...,2013 3rd round pick (STL - #83 - Bogdan Yakimo...,Sum: $0$0$0Change: $0Change: $0Change: $0,2013 2nd round pick (LAK - #57 - William Carrier),Sum: $0$0$0Change: $0Change: $0Change: $0
4,4,"Jun. 30, 2013",Chicago Blackhawks Acquire:2013 3rd round pick...,"Winnipeg Jets Acquire:Michael Frolík · $2,333,...",Chicago Blackhawks,2013 3rd round pick (WPG - #74 - John Hayden)2...,Winnipeg Jets,"Michael Frolík · $2,333,333$2,333,333$1,900,00...",2013 3rd round pick (WPG - #74 - John Hayden)2...,"Sum: $0$0$0Change: -$2,333,333Change: -$2,333,...",Michael Frolík,"· $2,333,333$2,333,333$1,900,000Sum: $2,333,3..."


Alright ton of progress made, definitely not 100% correct but some manual labor is going to be needed in addition to the web scraping. But, the web scraping saves a ton of time so going to move forward with it for other seasons to build out and help finish the rest of the trade datasets for other seasons. 

To finish the coding aspect of the trades that occured during the 2022-23 season, I am going to delete the extra columns. From there, I will finish this dataset completely by manually editing some of the data in the csv file itslef. 

In [13]:
better_df.head(2)

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary,receivedy,Monetary2
0,0,"Jun. 30, 2013","Minnesota Wild Acquire:Benn Ferriero · $0$550,...","New York Rangers Acquire:Justin Falk · $975,00...",Minnesota Wild,"Benn Ferriero · $0$550,000$550,000 (AHL/JR)201...",New York Rangers,"Justin Falk · $975,000$975,000$975,000Sum: $97...",Benn Ferriero,"· $0$550,000$550,000 (AHL/JR)2014 6th round p...",Justin Falk,"· $975,000$975,000$975,000Sum: $975,000$975,0..."
1,1,"Jun. 30, 2013",Columbus Blue Jackets Acquire:2013 2nd round p...,Pittsburgh Penguins Acquire:2013 2nd round pic...,Columbus Blue Jackets,2013 2nd round pick (SJS - #50 - Dillon Heathe...,Pittsburgh Penguins,2013 2nd round pick (CBJ - #44 - Tristan Jarry...,2013 2nd round pick (SJS - #50 - Dillon Heathe...,Sum: $0$0$0Change: $0Change: $0Change: $0,2013 2nd round pick (CBJ - #44 - Tristan Jarry),Sum: $0$0$0Change: $0Change: $0Change: $0


In [14]:
Final_df= better_df[['DATE','TRADE DETAILS', 'TRADE DETAILS.18','Team_x','receivedx','Team_y','receivedy']]


In [15]:
Final_df.to_csv("NHL 2012-13 Trade Data.csv")

Just a coding note:
To replace part of a string inside a variable of a dataset do the following: 
better_df['part1'] = better_df['part1'].str.replace(':', '')